In [13]:
import numpy as np

def gather_from_detector():
    return np.random.random((1000, 1000))

#@numba.njit
def smooth(x):
    out = np.zeros(x.shape)
    for i in range(1, x.shape[0]-1):
        for j in range(1, x.shape[1]-1):
            out[i, j] = (x[i-1, j-1] + x[i-1, j] + x[i-1, j+1] +
                         x[i,   j-1] + x[i,   j] + x[i,   j+1] +
                         x[i+1, j-1] + x[i+1, j] + x[i+1, j+1]) // 9
    return out

def save(x, filename):
    pass

In [2]:
%%time 

for i in range(5):
    print(i)
    img = gather_from_detector()
    img = smooth(img)
    img = np.fft.fft2(img)
    save(img, "file-"+str(i)+".dat")

0
1
2
3
4
CPU times: user 13.3 s, sys: 72.9 ms, total: 13.3 s
Wall time: 13.4 s


In [3]:
import numba

fast_smooth = numba.njit(smooth)

In [14]:
img = gather_from_detector()

In [15]:
img

array([[0.94124751, 0.77251403, 0.48613765, ..., 0.64027263, 0.97604334,
        0.9897515 ],
       [0.45473902, 0.31638288, 0.15492992, ..., 0.98461535, 0.24301047,
        0.83053084],
       [0.93144218, 0.22317878, 0.82882159, ..., 0.63675627, 0.06354869,
        0.11350888],
       ...,
       [0.35587939, 0.90028845, 0.4273799 , ..., 0.27563039, 0.20928773,
        0.26543247],
       [0.65922688, 0.99752101, 0.89794241, ..., 0.16953549, 0.84406439,
        0.04122452],
       [0.15535486, 0.95566572, 0.46655086, ..., 0.91784259, 0.81957754,
        0.22466055]])

In [16]:
%%time

img1 = smooth(img)

CPU times: user 2.74 s, sys: 17.2 ms, total: 2.75 s
Wall time: 2.76 s


In [17]:
img1

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [7]:
%time fast_smooth(img)

CPU times: user 8.67 ms, sys: 228 µs, total: 8.9 ms
Wall time: 9.01 ms


array([[ 1.11303118e+01,  6.73272812e+00,  7.09916631e-01, ...,
         1.50371933e+00, -7.35559316e+00, -9.70829462e-01],
       [-5.12764101e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  3.64785866e-03],
       [ 4.39576935e-01,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00, -3.64785866e-03],
       ...,
       [-8.86298922e-02,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  1.81815071e-02],
       [ 1.28670666e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00, -1.81815071e-02],
       [-6.41916129e-01, -2.42408929e-02, -6.41744785e-01, ...,
         7.40104123e-03,  9.89360129e-01,  1.11011966e-02]])

In [1]:
from dask.distributed import Client, LocalCluster, fire_and_forget

cluster = LocalCluster()
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:62891,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:62906,Total threads: 2
Dashboard: http://127.0.0.1:62909/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:62894,


In [4]:
for i in range(100):
    img = client.submit(gather_from_detector, pure=False)
    img = client.submit(smooth, img)
    img = client.submit(np.fft.fft2, img)
    future = client.submit(save, img, "file-"+str(i)+".dat")
    fire_and_forget(future)
    

In [3]:
cluster.scale(4)

In [ ]:
import asyncio

async def gather():
    while True:
        img = client.submit(gather_from_detector, pure=False)
        img = client.submit(smooth, img)
        img = client.submit(np.fft.fft2, img)
        future = client.submit(save, img, "file-"+str(i)+".dat")
        fire_and_forget(future)
        await asyncio.sleep(0.01)

asyncio.ensure_future(gather())